# 加载数据集


In [4]:
from scipy.io import loadmat
import numpy as np

file_path = 'dataset/ADNI_90_120_fMRI.mat'

labels=['AD', 'EMCI', 'LMCI', 'NC']

label_mapping = {
    "AD": 0,
    "EMCI": 1,
    "LMCI": 2,
    "NC": 3,
}

# 使用loadmat函数读取.mat文件
data = loadmat(file_path)

# data格式
data={  
    'AD':array(59, 90, 120),  
    'EMCI':array(56, 90, 120),  
    'LMCI':array(43, 90, 120),  
    'NC':array(48, 90, 120),  
}
# 转换为
data:(206,90,120)  
label:(206,)

尚未归一化  
后续：对于 不同样本间的 同一脑区的 同一时间的数据 进行归一化

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler


ad = data['AD']
emci = data['EMCI']
lmci = data['LMCI']
nc = data['NC']


all_labels = np.concatenate((np.zeros(ad.shape[0]), np.ones(emci.shape[0]), np.full(lmci.shape[0], 2), np.full(nc.shape[0], 3)))

data_combined = np.vstack((ad, emci, lmci, nc))

data_combined.shape

# 分为训练和验证

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data_combined, all_labels, test_size=0.2, random_state=42)

展示一个样本

In [ ]:
d=data_combined[0][0]

plt.scatter(range(len(d)), d)
plt.xlabel('Index')

# RNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Dense, Reshape, Bidirectional, Dropout,BatchNormalization, Activation
from tensorflow.keras.regularizers import l2


model = Sequential()
model.add(Reshape((90, 120), input_shape=(90, 120, 1)))

model.add(GRU(32, return_sequences= True,kernel_regularizer=l2(0.1)))
model.add(Dropout(0.5))
model.add(Activation('softmax'))
model.add(BatchNormalization())

model.add(GRU(32, return_sequences=False,kernel_regularizer=l2(0.1)))
model.add(BatchNormalization())
model.add(Activation('softmax'))


model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [25]:
# 学习率调度器
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.000000001)

In [ ]:
from tensorflow.keras.utils import to_categorical


y_train_one_hot = to_categorical(y_train, num_classes=4)
y_val_one_hot = to_categorical(y_val, num_classes=4)

history = model.fit(
    X_train, y_train_one_hot, 
    validation_data=(X_val, y_val_one_hot), 
    epochs=50,
    batch_size=64,
    callbacks=[reduce_lr]
)

In [ ]:
# 绘制训练和验证的损失和准确率曲线
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()